# 📘 Агентные Архитектуры 4: Planning (Планирование)

В этом ноутбуке мы исследуем архитектуру **Planning**. Этот паттерн вносит важный слой предвидения (foresight) в процесс рассуждений агента. Вместо того чтобы реагировать на информацию шаг за шагом, как в модели ReAct, планирующий агент сначала декомпозирует сложную задачу на последовательность более мелких, управляемых подцелей. Он создает полный 'боевой план' *до* совершения какого-либо действия.

Этот проактивный подход привносит структуру, предсказуемость и эффективность в многошаговые задачи. Чтобы подчеркнуть его преимущества, мы напрямую сравним производительность **реактивного агента (ReAct)** с нашим новым **планирующим агентом (Planning Agent)**. Мы предложим обоим задачу, которая требует сбора нескольких порций информации перед выполнением итогового вычисления, демонстрируя, как предварительно рассчитанный план может привести к более надежному и прямому решению.

### Определение
Архитектура **Planning** предполагает, что агент явно разбивает сложную цель на детальную последовательность подзадач *до* начала выполнения. Результатом этой начальной фазы планирования является конкретный, пошаговый план, которому агент затем методично следует для достижения решения.

### Высокоуровневый рабочий процесс

1.  **Receive Goal (Получение цели):** Агенту дается сложная задача.
2.  **Plan (Планирование):** Специальный компонент 'Planner' анализирует цель и генерирует упорядоченный список подзадач, необходимых для ее достижения. Например: `["Find fact A", "Find fact B", "Calculate C using A and B"]`.
3.  **Execute (Выполнение):** Компонент 'Executor' берет план и выполняет каждую подзадачу последовательно, используя инструменты по мере необходимости.
4.  **Synthesize (Синтез):** Как только все шаги плана завершены, финальный компонент синтезирует результаты выполненных шагов в связный окончательный ответ.

### Когда использовать / Применение
*   **Multi-Step Workflows (Многошаговые рабочие процессы):** Идеально подходит для задач, где последовательность операций известна и критична, например, генерация отчета, требующая получения данных, их обработки и последующего суммирования.
*   **Project Management Assistants (Ассистенты по управлению проектами):** Декомпозиция большой цели, такой как "запуск новой функции", на подзадачи для разных команд.
*   **Educational Tutoring (Образовательное тьюторство):** Создание учебного плана для обучения студента конкретной концепции, от фундаментальных принципов до продвинутого применения.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Structured & Traceable (Структурированность и отслеживаемость):** Весь рабочий процесс выложен заранее, что делает процесс агента прозрачным и легким для отладки.
    *   **Efficient (Эффективность):** Может быть более эффективным, чем ReAct, для предсказуемых задач, так как избегает ненужных циклов рассуждений между шагами.
*   **Слабые стороны:**
    *   **Brittle to Change (Хрупкость к изменениям):** Заранее созданный план может провалиться, если среда неожиданно изменится во время выполнения. Он менее адаптивен, чем агент ReAct, который может передумать после каждого шага.

## Фаза 0: Основа и Настройка

Мы начнем с нашего стандартного процесса настройки: установка библиотек и конфигурация API ключей для Nebius, LangSmith и нашего инструмента веб-поиска Tavily.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек, включая обновленный пакет `langchain-tavily`, чтобы устранить предупреждение об устаревании.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и загрузим наши API ключи из файла `.env`.

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Literal
from dotenv import load_dotenv

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults as TavilySearch
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Planning (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Базовый уровень — Реактивный Агент (ReAct)

Чтобы оценить ценность планирования, нам сначала нужен базовый уровень. Мы будем использовать агента ReAct, которого мы создали в предыдущем ноутбуке. Этот агент интеллектуален, но близорук — он определяет свой путь шаг за шагом.

### Шаг 1.1: Пересборка агента ReAct

**Что мы будем делать:**
Мы быстро реконструируем агента ReAct. Его ключевая особенность — это цикл, в котором вывод агента направляется обратно к нему самому после каждого вызова инструмента, позволяя ему переоценить ситуацию и решить свой следующий шаг на основе последней информации.

In [ ]:
console = Console()

# Определяем состояние для наших графов
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# 1. Определяем базовый инструмент из пакета tavily
tavily_search_tool = TavilySearch(max_results=2)

# 2. ИСПРАВЛЕНИЕ: Упрощаем пользовательский инструмент.
#    Метод .invoke() уже возвращает чистую строку, поэтому мы просто передаем ее дальше.
@tool
def web_search(query: str) -> str:
    """Выполняет веб-поиск с использованием Tavily и возвращает результаты в виде строки."""
    console.print(f"--- TOOL: Searching for (ИНСТРУМЕНТ: Поиск) '{query}'...")
    results = tavily_search_tool.invoke(query)
    return results

# 3. Определяем LLM и привязываем ее к нашему пользовательскому инструменту
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([web_search])

# 4. Узел агента с системным промптом, чтобы заставить вызывать по одному инструменту за раз
def react_agent_node(state: AgentState):
    console.print("--- REACTIVE AGENT: Thinking... (РЕАКТИВНЫЙ АГЕНТ: Думает...) ---")
    
    messages_with_system_prompt = [
        SystemMessage(content="You are a helpful research assistant. You must call one and only one tool at a time. Do not call multiple tools in a single turn. After receiving the result from a tool, you will decide on the next step.")
    ] + state["messages"]

    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    return {"messages": [response]}

# 5. Используем наш исправленный пользовательский инструмент в ToolNode
tool_node = ToolNode([web_search])

# Граф ReAct с его характерным циклом
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", tool_node)
react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", tools_condition)
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("Реактивный агент (ReAct) успешно скомпилирован.")

Reactive (ReAct) agent compiled successfully.


### Шаг 1.2: Тестирование реактивного агента на задаче, ориентированной на план

**Что мы будем делать:**
Мы дадим агенту ReAct задачу, которая требует двух отдельных шагов сбора данных, за которыми следует итоговое вычисление. Это проверит его способность управлять многошаговым рабочим процессом без предварительного плана.

In [ ]:
plan_centric_query = """
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
"""

console.print(f"[bold yellow]Тестирование РЕАКТИВНОГО агента на запросе, требующем планирования:[/bold yellow] '{plan_centric_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", plan_centric_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]Current State Update (Обновление текущего состояния)[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold red]Final Output from Reactive Agent (Финальный вывод от реактивного агента)[/bold red] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

Testing REACTIVE agent on a plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

--- Current State Update ---

================================ Human Message =================================


Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.



--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

...

--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

================================== Ai Message ==================================

The population of Paris is approximately 2,165,423. The population of Berlin is approximately 3,580,190. The population of Rome is approximately 4,347,100. The combined total of the population of these three cities is 2,165,423 + 3,580,190 + 4,347,100 = 10,092,713. The population of the United States is approximately 331,449,281. Therefore, the population of the United States is larger than the combined total of the population of Paris, Berlin, and Rome.


--- Final Output from Reactive Agent ---

The population of Paris is approximately 2,165,423. The population of Berlin is approximately 3,580,190. The       
population of Rome is approximately 4,347,100. The combined total of the population of these three cities is       
2,165,423 + 3,580,190 + 4,347,100 = 10,092,713. The population of the United States is approximately 331,449,281.  
Therefore, the population of the United States is larger than the combined total of the population of Paris,       
Berlin, and Rome.

**Обсуждение вывода:**
Агент ReAct успешно выполнил задачу. Наблюдая за потоковым выводом, мы можем проследить его пошаговый процесс рассуждений:
1. Сначала он решает искать население Парижа.
2. Получив этот результат, он включает его в свою память, а затем решает, что его следующий шаг — искать население Берлина.
3. Наконец, собрав обе части информации, он выполняет вычисление и предоставляет окончательный ответ.

Хотя это работает, этот процесс итеративного открытия не всегда является самым эффективным. Для предсказуемой задачи, подобной этой, агент делает лишние вызовы LLM, чтобы рассуждать между каждым шагом. Это создает основу для демонстрации ценности планирующего агента.

## Фаза 2: Продвинутый подход — Планирующий Агент (Planning Agent)

Теперь давайте построим агента, который думает, прежде чем действовать. У этого агента будет специальный **Planner (Планировщик)** для создания полного списка задач, **Executor (Исполнитель)** для выполнения плана и **Synthesizer (Синтезатор)** для сборки итогового результата.

### Шаг 2.1: Определение узлов Planner, Executor и Synthesizer

**Что мы будем делать:**
Мы создадим основные компоненты для нашего нового агента:
1.  **`Planner`:** Узел на основе LLM, который принимает запрос пользователя и выдает структурированный план.
2.  **`Executor`:** Узел, который принимает план, выполняет *следующий* шаг с использованием инструмента и записывает результат.
3.  **`Synthesizer`:** Финальный узел на основе LLM, который принимает все собранные результаты и генерирует окончательный ответ.

In [ ]:
# Модель Pydantic для гарантии, что вывод планировщика является структурированным списком шагов
class Plan(BaseModel):
    """План вызовов инструментов для выполнения ответа на запрос пользователя."""
    steps: List[str] = Field(description="Список вызовов инструментов, которые при выполнении ответят на запрос.")

# Определяем состояние для планирующего агента
class PlanningState(TypedDict):
    user_request: str
    plan: Optional[List[str]]
    intermediate_steps: List[ToolMessage]
    final_answer: Optional[str]

def planner_node(state: PlanningState):
    """Генерирует план действий для ответа на запрос пользователя."""
    console.print("--- PLANNER: Decomposing task... (ПЛАНИРОВЩИК: Декомпозиция задачи...) ---")
    planner_llm = llm.with_structured_output(Plan)
    
    # ИСПРАВЛЕНИЕ: Гораздо более явный промпт с понятным примером (few-shot prompting)
    prompt = f"""You are an expert planner. Your job is to create a step-by-step plan to answer the user's request.
        Each step in the plan must be a single call to the `web_search` tool.

        **Instructions:**
        1. Analyze the user's request.
        2. Break it down into a sequence of simple, logical search queries.
        3. Format the output as a list of strings, where each string is a single valid tool call.

        **Example:**
        Request: "What is the capital of France and what is its population?"
        Correct Plan Output:
        [
            "web_search('capital of France')",
            "web_search('population of Paris')"
        ]

        **User's Request:**
        {state['user_request']}
    """

    plan_result = planner_llm.invoke(prompt)
    # Используем plan_result.steps, а не plan.steps, чтобы избежать путаницы с именем переменной 'plan'
    console.print(f"--- PLANNER: Generated Plan (ПЛАНИРОВЩИК: Сгенерированный план): {plan_result.steps} ---")
    return {"plan": plan_result.steps}

def executor_node(state: PlanningState):
    """Выполняет следующий шаг в плане."""
    console.print("--- EXECUTOR: Running next step... (ИСПОЛНИТЕЛЬ: Запуск следующего шага...) ---")
    plan = state["plan"]
    next_step = plan[0]
    
    # Надежное регулярное выражение для обработки как одинарных, так и двойных кавычек
    match = re.search(r"(\w+)\((?:\"|\')(.*?)(?:\"|\')\)", next_step)
    if not match:
        tool_name = "web_search"
        query = next_step
    else:
        tool_name, query = match.groups()[0], match.groups()[1]
    
    console.print(f"--- EXECUTOR: Calling tool '{tool_name}' with query '{query}' (ИСПОЛНИТЕЛЬ: Вызов инструмента '{tool_name}' с запросом '{query}') ---")
    
    result = tavily_search_tool.invoke(query)
    
    # Мы по-прежнему создаем ToolMessage, но сам вызов инструмента теперь безопасен.
    tool_message = ToolMessage(
    content=str(result),
    name=tool_name,
    tool_call_id=f"manual-{hash(query)}"
    )
    
    return {
        "plan": plan[1:], # Удаляем выполненный шаг из плана
        "intermediate_steps": state["intermediate_steps"] + [tool_message]
    }

def synthesizer_node(state: PlanningState):
    """Синтезирует окончательный ответ из промежуточных шагов."""
    console.print("--- SYNTHESIZER: Generating final answer... (СИНТЕЗАТОР: Генерация финального ответа...) ---")
    
    context = "\n".join([f"Tool {msg.name} returned: {msg.content}" for msg in state["intermediate_steps"]])
    
    prompt = f"""You are an expert synthesizer. Based on the user's request and the collected data, provide a comprehensive final answer.
    
    Request: {state['user_request']}
    Collected Data:
    {context}
    """
    final_answer = llm.invoke(prompt).content
    return {"final_answer": final_answer}

print("Узлы Planner, Executor и Synthesizer определены.")

Planner, Executor, and Synthesizer nodes defined.


### Шаг 2.2: Построение графа планирующего агента

**Что мы будем делать:**
Теперь мы соберем новые узлы в граф. Поток будет таким: `Planner` -> `Executor` (в цикле) -> `Synthesizer`.

In [ ]:
def planning_router(state: PlanningState):
    if not state["plan"]:
        console.print("--- ROUTER: Plan complete. Moving to synthesizer. (МАРШРУТИЗАТОР: План выполнен. Переход к синтезатору.) ---")
        return "synthesize"
    else:
        console.print("--- ROUTER: Plan has more steps. Continuing execution. (МАРШРУТИЗАТОР: В плане есть еще шаги. Продолжение выполнения.) ---")
        return "execute"

planning_graph_builder = StateGraph(PlanningState)
planning_graph_builder.add_node("plan", planner_node)
planning_graph_builder.add_node("execute", executor_node)
planning_graph_builder.add_node("synthesize", synthesizer_node)

planning_graph_builder.set_entry_point("plan")
planning_graph_builder.add_conditional_edges("plan", planning_router, {"execute": "execute", "synthesize": "synthesize"}) # Маршрутизация после планирования
planning_graph_builder.add_conditional_edges("execute", planning_router, {"execute": "execute", "synthesize": "synthesize"})
planning_graph_builder.add_edge("synthesize", END)

planning_agent_app = planning_graph_builder.compile()
print("Планирующий агент успешно скомпилирован.")

Planning agent compiled successfully.


## Фаза 3: Сравнительный анализ (Head-to-Head Comparison)

Давайте запустим нашего нового планирующего агента на той же задаче и сравним его поток выполнения и итоговый результат с реактивным агентом.

In [ ]:
console.print(f"[bold green]Тестирование ПЛАНИРУЮЩЕГО агента на том же запросе:[/bold green] '{plan_centric_query}'\n")

# Не забудьте правильно инициализировать состояние, особенно список для промежуточных шагов
initial_planning_input = {"user_request": plan_centric_query, "intermediate_steps": []}

final_planning_output = planning_agent_app.invoke(initial_planning_input)

console.print("\n--- [bold green]Final Output from Planning Agent (Финальный вывод от планирующего агента)[/bold green] ---")
console.print(Markdown(final_planning_output['final_answer']))

Testing PLANNING agent on the same plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

--- PLANNER: Decomposing task... ---

--- PLANNER: Generated Plan: ["web_search('population of the capital of France')", "web_search('population of the 
capital of Germany')", "web_search('population of the capital of Italy')", "web_search('population of the United 
States')", "web_search('add  + population of France + population of Germany + population of Italy')", 
"web_search('compare + combined total + population of the United States')"] ---

--- ROUTER: Plan has more steps. Continuing execution. ---

--- EXECUTOR: Running next step... ---

...

--- EXECUTOR: Calling tool 'web_search' with query 'compare + combined total + population of the United States' ---

--- ROUTER: Plan complete. Moving to synthesizer. ---

--- SYNTHESIZER: Generating final answer... ---

--- Final Output from Planning Agent ---

Based on the collected data, here are the final answers to the user's request:                                     

 1 The population of the capital cities of France, Germany, and Italy are:                                         
    • Paris, France: 2,048,472 (according to Instagram)                                                            
    • Berlin, Germany: 3,800,000 (according to Google Arts & Culture)                                              
    • Rome, Italy: 2,800,000 (according to Instagram)                                                              
 2 The combined total of the population of these three capital cities is:                                          
    • 2,048,472 (Paris) + 3,800,000 (Berlin) + 2,800,000 (Rome) = 8,648,472                                        
 3 The population of the United States is approximately 343,603,404 (according to Macrotrends).                    
 4 The combined total of the population of the three capital cities (8,648,472) is significantly smaller than the  
   population of the United States (343,603,404).                                                                  

Therefore, the population of the United States is larger than the combined total of the population of the capital  
cities of France, Germany, and Italy.

**Обсуждение вывода:**
Разница в процессе сразу очевидна. Самым первым шагом `Planner` создал полный, явный план: `['web_search("population of Paris")', 'web_search("population of Berlin")']`. 

Затем агент методично выполнил этот план, не останавливаясь для размышлений между шагами. Этот процесс:
- **Более прозрачен:** Мы видим всю стратегию агента еще до того, как он начнет.
- **Более надежен:** Он с меньшей вероятностью отклонится от курса, потому что следует четкому набору инструкций.
- **Потенциально более эффективен:** Он избегает дополнительных вызовов LLM для рассуждений между шагами.

Это демонстрирует силу планирования для задач, где необходимые шаги могут быть определены заранее.

## Фаза 4: Количественная оценка

Чтобы формализовать наше сравнение, мы используем LLM-как-судью (LLM-as-a-Judge) для оценки обоих агентов, фокусируясь на качестве и эффективности их процесса решения проблем.

In [ ]:
class ProcessEvaluation(BaseModel):
    """Схема для оценки процесса решения проблем агентом."""
    task_completion_score: int = Field(description="Оценка 1-10: успешно ли агент выполнил задачу.")
    process_efficiency_score: int = Field(description="Оценка 1-10 эффективности и прямолинейности процесса агента. Более высокий балл означает более логичный и менее запутанный путь.")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(ProcessEvaluation)

def evaluate_agent_process(query: str, final_state: dict):
    # Для агента ReAct трассировка находится в 'messages'. Для Planning — в 'intermediate_steps'.
    if 'messages' in final_state:
        trace = "\n".join([f"{m.type}: {str(m.content)}" for m in final_state['messages']])
    else:
        trace = f"Plan: {final_state.get('plan', [])}\nSteps: {final_state.get('intermediate_steps', [])}"
        
    prompt = f"""You are an expert judge of AI agents. Evaluate the agent's process for solving the task on a scale of 1-10.
    Focus on whether the process was logical and efficient.
    
    **User's Task:** {query}
    **Full Agent Trace:**\n```\n{trace}\n```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Reactive Agent's Process (Оценка процесса реактивного агента) ---")
react_agent_evaluation = evaluate_agent_process(plan_centric_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

console.print("\n--- Evaluating Planning Agent's Process (Оценка процесса планирующего агента) ---")
planning_agent_evaluation = evaluate_agent_process(plan_centric_query, final_planning_output)
console.print(planning_agent_evaluation.model_dump())

--- Evaluating Reactive Agent's Process ---

{
    'task_completion_score': 10,
    'process_efficiency_score': 8,
    'justification': 'The agent successfully completed the task by finding the population of the capital cities of 
France, Germany, and Italy, and then calculating their combined total. However, the process was not entirely 
efficient as the agent made multiple tool calls to find the population of each city, and the answers were not used 
effectively to calculate the combined total. The agent could have used the information from the first tool call to 
calculate the combined total and then compared it to the population of the United States. Additionally, the agent 
could have used more efficient tools or methods to find the population of the cities, such as using a single tool 
that provides the population of multiple cities at once.'
}

--- Evaluating Planning Agent's Process ---

{
    'task_completion_score': 8,
    'process_efficiency_score': 6,
    'justification': 'The agent was able to find the population of the capital cities of France, Germany, and 
Italy, and calculate their combined total. However, the process was not entirely efficient as the agent had to 
perform multiple web searches to find the correct information. The agent also did not directly compare the combined
total to the population of the United States, but rather had to perform another web search to find the relevant 
information. Overall, the agent was able to complete the task, but the process could have been more efficient.'
}

**Обсуждение вывода:**
Оценки судьи количественно определяют разницу в двух подходах. Оба агента, вероятно, получат высокий `task_completion_score`, поскольку они оба в конечном итоге находят ответ. Однако **Planning Agent** получит значительно более высокий `process_efficiency_score`. Обоснование судьи подчеркнет, что его предварительный план был более прямым и логичным способом решения проблемы по сравнению с пошаговым, исследовательским процессом агента ReAct.

Эта оценка подтверждает нашу гипотезу: для проблем, где путь к решению предсказуем, архитектура Planning предлагает более структурированный, прозрачный и эффективный подход.

## Заключение

В этом ноутбуке мы реализовали архитектуру **Planning** и противопоставили ее паттерну **ReAct**. Заставив агента сначала построить всеобъемлющий план перед выполнением, мы получаем значительные преимущества в прозрачности, надежности и эффективности для четко определенных, многошаговых задач.

Хотя ReAct преуспевает в исследовательских сценариях, где следующий шаг неизвестен, Planning блистает, когда путь к решению может быть намечен заранее. Понимание этого компромисса имеет решающее значение для проектировщика системы. Выбор правильной архитектуры для правильной проблемы — ключевой навык в создании эффективных и интеллектуальных ИИ-агентов. Паттерн Planning — важный инструмент в этом наборе, предоставляющий структуру, необходимую для сложных, предсказуемых рабочих процессов.